In [67]:
# all import statements needed
import os
import json
import requests
import shutil
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv

In [68]:
# get planet api key from .env
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

In [69]:
# set the location that we are interested in
region =  {
        "type": "Polygon",
        "coordinates": [
          [
            [-158.40087890625,70.57976496276407],
            [-154.0887451171875,70.57976496276407],
            [-154.0887451171875,71.38338855861728],
            [-158.40087890625,71.38338855861728],
            [-158.40087890625, 70.57976496276407]
          ]
        ]
      }


In [70]:
# locations that overlap region of interest
location = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": region
}
# select dates for summer months
dates = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-05-01T00:00:00.000Z",
    "lte": "2016-10-01T00:00:00.000Z"
  }
}
# cloud cover of less than 0.1
cloud_cover= {
    "type": "RangeFilter",
    "field_name":"cloud_cover",
    "config":{
      "lte": 0.1
    }
}
# create a filter json that combines all of the filters that have been defined
filters = {
    "type": "AndFilter",
    "config": [location, dates, cloud_cover]
}


In [71]:
# set the item type that we are insterested
item_type = 'PSScene4Band'

# create a request object
search_request = {
    "item_types":[item_type],
    "filter":filters
}

In [72]:
# send request
search_result = \
    requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request
    )

In [73]:
print(json.dumps(search_result.json(), indent=1))

{
 "_links": {
  "_first": "https://api.planet.com/data/v1/searches/ad49297f1b6f4adebee10b3d707be451/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D",
  "_next": "https://api.planet.com/data/v1/searches/ad49297f1b6f4adebee10b3d707be451/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDE2LTA5LTE0VDEyOjIzOjQ5LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTYwOTA4XzIxMjkzMF8wZTFmIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319",
  "_self": "https://api.planet.com/data/v1/searches/ad49297f1b6f4adebee10b3d707be451/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D"
 },
 "features": [
  {
   "_links": {
    "

In [74]:
# filter only the images
images = [feature['id'] for feature in search_result.json()['features']]

In [ ]:
# get all links to images from
for i, image in enumerate(images):
    image_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, image)
    result = \
        requests.get(
            image_url,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )
    links = result.json()[u"analytic"]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]
    # activate links from
    activate_result = \
        requests.get(
            activation_link,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )
    activation_status_result = \
        requests.get(
            self_link,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )
    while (activation_status_result.json()["status"] != "active"):
        activation_status_result = \
        requests.get(
            self_link,
            auth=HTTPBasicAuth(PLANET_API_KEY, '')
        )
        print(activation_status_result.json()["status"])
    # print('here is the download link')
    # downloads.append(activation_status_result.json()["location"])
    # print(activation_status_result.json()["location"]) 
    url = activation_status_result.json()["location"]
    name = "images/image" + str(i) +".tif"
    download = requests.get(url, stream=True)
    download.raw.decode_content = True
    with open(name, "wb") as f:
        shutil.copyfileobj(download.raw, f)

